<a href="https://colab.research.google.com/github/edchan7-sama/sama-api-colabs/blob/main/Task_Deliveries_with_raster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Compatible with Google Chrome only.

# Step 1 - Extract Latest Deliveries

## Input project settings and run

In [ ]:
#@title Step 1 - Set Project Settings { run: "auto", display-mode: "form" }
project_id = "" #@param {type:"string"}
client_access_key = "" #@param {type:"string"}
delivery_datetime = "2021-02-02" #@param {type:"date"}
delivery_datetime += "T00:00:00Z"
optional_batch_ids = "" #@param {type:"string"}
optional_client_batch_ids = "" #@param {type:"string"}


import requests 
import json


# Query URL
url = f"https://api.sama.com/api/v2/projects/{project_id}/tasks/delivered.json"

page = 1; page_size = 1000; tasks = []

while True:
  try:
    querystring = {"from":delivery_datetime,
                  "page":page,
                  "page_size":page_size,
                  "access_key":client_access_key,
                  "batch_id":optional_batch_ids.replace(" ", ""),
                  "client_batch_id":optional_client_batch_ids.replace(" ", "")}

    headers = {"Accept": "application/json"}

    # Make a GET request to the URL
    response = requests.request("GET", url, headers=headers, params=querystring)

    # Parse `response.text` into JSON
    payload = response.json()

    if response.status_code == 200:
      if payload.get("tasks"):
        tasks += payload.get("tasks")
        page += 1
      elif page == 1:
        raise Exception(f"No task deliveries for project {project_id} from {delivery_datetime}")
      else:
        raise Exception(f"Task deliveries available for project {project_id} from {delivery_datetime}")
    else:
      raise Exception(f"Error: Invalid response code, {response.status_code} : '{response.reason}'")
  except ValueError as e:
    print("optional_batch_ids input does not have valid integers")
    break
  except Exception as e:
    print(str(e))
    break

# Step 2 - Download files

## Option 1 - Download latest deliveries results as JSON


In [ ]:
from google.colab import files

if tasks:
  with open(f"{project_id}_{delivery_datetime}.json", "w") as f:
    json.dump(tasks, f, indent=4)
  files.download(f"{project_id}_{delivery_datetime}.json")
  print(f"Downloaded: {project_id}_{delivery_datetime}.json")


## Option 2 - Download latest deliveries results as CSV

In [ ]:
!pip install flatten_json --quiet
import pandas as pd
from flatten_json import flatten

# answer objects are nested JSONs, we want to keep them that way. 

try:
  if not tasks:
    raise Exception(f"No task deliveries for project {project_id} from {delivery_datetime}")

  # create a DataFrame from the task objects, but ignore answers as they should be kept as a dict 
  dic_flattened = [flatten(d, root_keys_to_ignore={'answers', 'data'}) for d in tasks]
  df = pd.DataFrame(dic_flattened)

  # create a DataFrame of just the answer objects, but convert to json
  answers = []
  for d in tasks:
      a = {**d.get('data'), **{ key: json.dumps(d.get('answers')[key]) for key in d.get('answers') } }
      a['id'] = d.get('id')
      answers.append(a)
  df_answers = pd.DataFrame(answers)

  csv_name = f"{project_id}_{delivery_datetime}.csv"
  # merge DataFrames and write to csv
  pd.merge(df, df_answers, on='id').to_csv(csv_name)
  files.download(csv_name)

except NameError:
  print("Run Get Delivered Tasks First")
except Exception as e:
  print(str(e))


## Option 3 - Download Raster Masks from latest deliveries

### Step 1 - Run and get individual files

In [ ]:
!pip install boto3
!mkdir masks

import boto3
import botocore
from botocore.config import Config
import json
import sys
from google.colab import files

# Download s3 files and show progress
def download(local_file_name, s3_bucket, s3_object_key):
    meta_data = s3_client.head_object(Bucket=s3_bucket, Key=s3_object_key)
    total_length = int(meta_data.get('ContentLength', 0))
    downloaded = 0

    def progress(chunk):
        nonlocal downloaded
        downloaded += chunk
        done = int(50 * downloaded / total_length)
        sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )
        sys.stdout.flush()

    print(f'\nDownloading {s3_object_key}')
    with open('masks/' + local_file_name, 'wb') as f:
        s3_client.download_fileobj(s3_bucket, s3_object_key, f, Callback=progress)

# Get AWS credentials to download masks
import requests

try:
  url = f"https://api.sama.com/v2/projects/{project_id}/credentials.json?access_key={client_access_key}"

  headers = {"Accept": "application/json"}

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    payload = response.json()
  else:
    raise Exception(f"Error: Invalid response code, {response.status_code} : '{response.reason}'")
except Exception as e:
    print(str(e))
    sys.exit()

# set up s3 boto with AWS credials
s3_client = boto3.client(
    's3',
    aws_access_key_id=payload.get('access_key_id'),
    aws_secret_access_key=payload.get('secret_access_key'),
    aws_session_token=payload.get('session_token')
)

json_array = tasks
for json_dict in json_array:
  for _output in json_dict['answers']:
    try:
      mask_url = json_dict.get('answers').get(_output).get('layers').get('raster_coding').get('mask_url')
      bucket = 'samahub3'
      s3_filename = mask_url.split("samahub3.s3.amazonaws.com/",1)[1]
      local_filename = mask_url.split('/')[-1]
      download(local_filename, bucket, s3_filename)
    except Exception as e:
      # answer object does not have a mask
      continue

### Step 2 - Zip all files and download

In [ ]:
!zip -r /content/masks.zip /content/masks
files.download(f"masks.zip")